**Import modules et données puis préparation**

In [1]:
import math
from random import randrange, randint
import numpy as np
import pandas as pd
df = pd.read_csv("info_communes_parc.csv", sep=";")

In [2]:
elite = 0.2
tx_mutations = 0.1
taille_echantillon = 5000
trajets = pd.DataFrame({"itineraire":[],"kms":[],"pois":[],"hotel_j3":[],"hotel_j7":[]})
nb_etapes = 10

In [3]:
def distance (depart, arrivee) :
    tmp = df.loc[df["insee"].isin([depart,arrivee]), "lat_centre":"long_centre"]
    x_dep = tmp.iloc[0,1]
    y_dep = tmp.iloc[0,0]
    x_arr = tmp.iloc[1,1]
    y_arr = tmp.iloc[1,0]
    distance = math.acos(math.sin(math.radians(x_dep))*math.sin(math.radians(x_arr))+math.cos(math.radians(x_dep))*math.cos(math.radians(x_arr))*math.cos(math.radians(y_dep-y_arr)))*6371
    return round(distance,1)
noms_villes = df["insee"].tolist()

In [4]:
def generateur_aleatoire (n_etapes, n_iterations, liste) :
    trajets = []
    for h in range(n_iterations) :
        liste_voyage = []
        tmp_liste = liste.copy()
        for i in range(n_etapes) :
            nombre_alea = randrange(0, len(tmp_liste), 1)
            liste_voyage.append(tmp_liste[nombre_alea])
            tmp_liste.pop(nombre_alea)
        trajets.append(liste_voyage)
    return trajets

In [5]:
trajets["itineraire"] = generateur_aleatoire(nb_etapes,taille_echantillon, noms_villes)

In [50]:
def notation () :
    #pense bete tri colonnes array[array[:, 1].argsort()]
    global trajets
    #decompte distance
    for i in range(taille_echantillon) :
        distance_totale = 0
        nb_poi = 0
        print("ligne "+ str(i),end='\r')
        for j in range(9) :
            distance_totale += distance(trajets.iloc[i,0][j],trajets.iloc[i,0][j+1])
        trajets.iloc[i,1] = distance_totale
    #decompte POI
        for k in range(10) :
            valeur = df.loc[df["insee"] == trajets.iloc[i,0][k], "nb_poi"].iloc[0]
            nb_poi += valeur
        trajets.iloc[i,2] = nb_poi
        trajets.iloc[i,3] = df.loc[df["insee"] == trajets.iloc[i,0][2], "nb_hotels_3_etoiles_min"].iloc[0]
        trajets.iloc[i,4] = df.loc[df["insee"] == trajets.iloc[i,0][6], "nb_hotels_3_etoiles_min"].iloc[0]

In [53]:
notation()

In [54]:
trajets = trajets.sort_values(by="kms",ascending=True)
trajets["decile_km"] = 0
for i in range(trajets.shape[0]) :
    trajets.iloc[i,5] = math.floor(i / (taille_echantillon / 10)) + 1
trajets = trajets.sort_values(by="pois",ascending=False)
trajets["decile_poi"] = 0
for i in range(trajets.shape[0]) :
    trajets.iloc[i,6] = math.floor(i / (taille_echantillon / 10)) + 1
trajets["decile_cumul"] = 10
for i in range(trajets.shape[0]) :
    for j in range(1,10) :
        if trajets.iloc[i,5] == j and trajets.iloc[i,6] == j :
            trajets.iloc[i,7] = j
trajets = trajets.sort_values(by=["decile_cumul","pois","kms"],ascending=[True,False,True])

In [64]:
if trajets[trajets["decile_cumul"]!=10].shape[0] < math.floor(taille_echantillon * elite) :
    nextgen = trajets.iloc[0:trajets[trajets["decile_cumul"]!=10].shape[0],0].tolist()
else :
    nextgen = trajets.iloc[0:math.floor(taille_echantillon * elite),0].tolist()

In [59]:
def mutations (tx_mutation) :
    global trajets
    tmp_liste = trajets.sample(frac=tx_mutation).iloc[:,0].tolist()
    liste_mutants = []
    for genome in tmp_liste :
        alea = randint(0,2)
        if alea == 0 :
            gene1 = randrange(0,10)
            gene2 = gene1
            while gene2 == gene1 :
                gene2 = randrange(0,10)
            genome[gene1], genome[gene2] = genome[gene2], genome[gene1]
            liste_mutants.append(genome)
        elif alea == 1 :
            gene1 = randrange(0,10)
            gene2 = gene1
            while gene2 == gene1 :
                gene2 = randrange(0,10)
            gene_mutation = genome[gene2]
            del(genome[gene2])
            genome.insert(gene1 + 1, gene_mutation)
            liste_mutants.append(genome)
        else :
            debut = randrange(0,10)
            fin = randrange(debut,10)
            
        
    return tmp_liste

In [60]:
lesmutants = mutations(tx_mutations)

[71142, 58047, 71266, 21349, 89089, 58066, 89409, 58128, 58261, 58199]
[71142, 58047, 71266, 21349, 89089, 58066, 58128, 89409, 58261, 58199]
[89351, 58185, 58010, 58211, 58255, 58125, 58066, 89145, 21703, 58047]
[89351, 58185, 58255, 58010, 58211, 58125, 58066, 89145, 21703, 58047]
[71020, 21548, 58068, 58106, 71142, 89349, 21546, 71129, 58128, 58149]
[71020, 71142, 21548, 58068, 58106, 89349, 21546, 71129, 58128, 58149]
[58099, 58168, 21349, 58034, 58023, 58141, 21687, 58125, 58120, 21546]
[58099, 58168, 21349, 58125, 58034, 58023, 58141, 21687, 58120, 21546]
[21430, 89032, 58244, 21608, 89170, 21403, 58182, 71129, 71527, 58166]
[21430, 89032, 58244, 89170, 21403, 58182, 21608, 71129, 71527, 58166]
[71535, 21046, 58107, 58261, 89318, 58125, 58185, 58262, 21538, 71142]
[71535, 21046, 58107, 58261, 58125, 58185, 89318, 58262, 21538, 71142]
[58082, 58129, 71266, 58157, 21445, 89089, 58106, 58149, 58128, 21434]
[58082, 58106, 58129, 71266, 58157, 21445, 89089, 58149, 58128, 21434]
[58099